In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

# Analisis Exploratorio de Datos

## Carga de la tabla

In [2]:
calls_carga = pd.read_csv('Call_Center_1999.csv',delimiter = ';',encoding = "utf-8")

FileNotFoundError: [Errno 2] No such file or directory: 'Call_Center_1999.csv'

In [ ]:
calls_carga

## Normalizacion

In [12]:
calls = calls_carga.copy()

In [ ]:
calls

Se borraron las columnas vru_exit, q_exit, ser_exit, q_start, ser_start y startdate debido a que la informacion que aportaban esta columnas era redundante ya que se puede deducir de otras columnas (vru_entry, vru_time, q_time, ser_time).<br>

In [14]:
calls.drop(columns=['vru_exit', 'q_exit', 'ser_exit', 'q_start', 'ser_start', 'startdate'], inplace=True)

In [ ]:
calls

Se realizó el cambio de 'priority' de 0, 1 y 2 a "Normal" y "Alta" para identificar con mayor facilidad el nivel de prioridad de cada cliente<br>

In [16]:
calls['priority']=calls['priority'].replace(2,'Alta')
calls['priority']=calls['priority'].replace(1,'Normal')
calls['priority']=calls['priority'].replace(0,'Normal')

Se consideró que los nombres "ANO_SERVERT" y "NO_SERVERAMA" tuvieron un error de ingreso y correspondian a "NO_SERVER" por lo cual se realizó el cambio. Se modificaron 1221 registros.

In [17]:
calls['server']=calls['server'].replace('NO_SERVERAMA','NO_SERVER')
calls['server']=calls['server'].replace('ANO_SERVERT','NO_SERVER')

In [ ]:
calls.loc[calls['server']=='NO_SERVERAMA']
calls.loc[calls['server']=='ANO_SERVERT']

Se identificó que se cometió un error al digitar el valor 'TT' en 'type'. Se cambió el valor ' TT' por 'TT'. Se actualizaron 6 registros

In [19]:
calls['type']=calls['type'].replace(' TT','TT')

In [ ]:
calls.loc[calls['type']==' TT']

Al continuar con la verificación de los registros de llamadas, se indentificó que hay un tipo llamada categorizada como "AA" que no está presente en la categorización de los tipos de llamada en nuestra base de datos. 

In [ ]:
calls[calls['type']=='AA']

Solo tiene vru_time

Se borraron los registros de llamadas que tienen los tres campos: vru_time, q_time y ser_time en cero, ya que para el analisis a realizar la variable principal son los tiempos, por lo que no se consideran objeto de estudio y no aportan informacion relevante. 

5390 registros

In [21]:
calls.drop(calls[(calls['q_time']==0)&(calls['ser_time']==0)&(calls['vru_time']==0)].index, inplace=True)

In [ ]:
calls

## Analisis de Outliers en tiempos

### Tiempo en VRU

In [ ]:
calls['vru_time'].describe()

In [ ]:
plt.plot(calls['vru_time']/60,'o',markersize=0.8)
plt.title('Tiempo en VRU', fontweight = 'bold',fontsize=15)
plt.xlabel('Llamadas')
plt.ylabel('tiempo (min)')
plt.show()

#### Boxplot

Limites segun el Boxplot: tiempo maximo 16 segundos y minimo 0 segundos <br>

In [ ]:
#Calculo de bigotes
Q3 = float(calls['vru_time'].quantile(0.75))
Q1 = float(calls['vru_time'].quantile(0.25))
RIC = Q3 - Q1

Min_vru_bp = Q1 - 1.5*RIC
if Min_vru_bp < min(calls['vru_time']):
    Min_vru_bp = min(calls['vru_time'])
Max_vru_bp = Q3 + 1.5*RIC
if Max_vru_bp > max(calls['vru_time']):
    Max_vru_bp = max(calls['vru_time'])

Min_vru_bp, Max_vru_bp

In [ ]:
#Box plot completo
plt.boxplot(calls['vru_time']/60)
plt.title('Boxplot de Tiempo en VRU',fontsize=15,fontweight='bold')
plt.ylabel('tiempos (min)')
plt.show()

In [ ]:
#Box plot enfocado en el box
plt.boxplot(calls['vru_time']/60)
plt.ylim(Min_vru_bp*0.9/60,Max_vru_bp*1.1/60)
plt.title('Boxplot de Tiempo en VRU reducido',fontsize=15,fontweight='bold')
plt.ylabel('tiempos (min)')
plt.show()

#### Sigmas

1 sigma

Limites segun el metodo de 1 sigma: tiempo maximo 45.55 segundos y minimo -24.72 segundos

In [ ]:
Max_vru_s1 = calls['vru_time'].mean() + (1 * calls['vru_time'].std())
Min_vru_s1 = calls['vru_time'].mean() - (1 * calls['vru_time'].std())

if Min_vru_s1 < min(calls['vru_time']):
    Min_vru_s1 = min(calls['vru_time'])
if Max_vru_s1 > max(calls['vru_time']):
    Max_vru_s1 = max(calls['vru_time'])

Min_vru_s1, Max_vru_s1

2 sigmas

Limites segun el metodo de los 2 sigmas: tiempo maximo 80.69 segundos (1.34 minutos) y minimo -59.86 segundos (aprox -1 minuto)

In [ ]:
Max_vru_s2 = calls['vru_time'].mean() + (2 * calls['vru_time'].std())
Min_vru_s2 = calls['vru_time'].mean() - (2 * calls['vru_time'].std())

if Min_vru_s2 < min(calls['vru_time']):
    Min_vru_s2 = min(calls['vru_time'])
if Max_vru_s2 > max(calls['vru_time']):
    Max_vru_s2 = max(calls['vru_time'])

Min_vru_s2, Max_vru_s2

3 sigmas

Limites segun el metodo de 1 sigma: tiempo maximo 115.82 segundos (1.93 minutos) y minimo -94.99 segundos (-1.58 minutos)

In [ ]:
Max_vru_s3 = calls['vru_time'].mean() + (3 * calls['vru_time'].std())
Min_vru_s3 = calls['vru_time'].mean() - (3 * calls['vru_time'].std())

if Min_vru_s3 < min(calls['vru_time']):
    Min_vru_s3 = min(calls['vru_time'])
if Max_vru_s3 > max(calls['vru_time']):
    Max_vru_s3 = max(calls['vru_time'])

Min_vru_s3, Max_vru_s3

#### Conclusion

In [ ]:
#Grafica de los outliers
plt.plot(calls['vru_time'],'o', markersize = 0.8)
plt.axhline(y=Max_vru_s3, color='green', linestyle='--', lw=1)
plt.axhline(y=Max_vru_s2, color='purple', linestyle='--', lw=1)
plt.axhline(y=Max_vru_s1, color='r', linestyle='--', lw=1)
plt.axhline(y=Max_vru_bp, color='orange', linestyle='--', lw=1)
plt.axhline(y=calls['vru_time'].mean(), color='black', linestyle='--', lw=1.4)
plt.axhline(y=0, color='black', linestyle='-', lw=0.9)
plt.ylim(Min_vru_s3*0.5,Max_vru_s3*1.8)
plt.legend(['Llamadas','3 sigmas','2 sigmas','1 sigma','BoxPlot','Promedio'],loc="upper right")
plt.ylabel('Tiempo en segundos')
plt.xlabel('Registros de llamadas')
plt.title('Outliers Tiempo en VRU',fontsize=15,fontweight='bold')
plt.show()

Despues de utilizar los 4 metodos anteriores para identificar los outliers (el metodo de Box Plot y diferentes sigmas) y teniendo en cuenta los resultados arrojados, se decidió utilizar el método de 1 sigma el cual establece un tiempo maximo de 45.62 segundos debido a que este contiene los tiempos mas representativos de la muestra. Adicionalmente, se observó la existencia de tiempos negativos lo cual no corresponde a la realidad. Con base en lo anterior, se borraron 4913 registros (1.10%) <br>

In [32]:
calls.drop(calls[(calls['vru_time']<0)|(calls['vru_time']>Max_vru_s1)].index,inplace=True)

Quedaron 434145

### Tiempo en Cola

In [ ]:
calls['q_time'].describe()

In [ ]:
plt.plot(calls['q_time']/60,'o',markersize=0.8)
plt.title('Tiempo en Cola', fontweight = 'bold',fontsize=15)
plt.xlabel('Llamadas')
plt.ylabel('tiempo en minutos')
plt.show()

#### Boxplot

Limites segun el Boxplot: tiempo maximo 202.5 segundos (3.38 minutos) y minimo 0 segundos <br>

In [ ]:
#Calculo de bigotes
Q3 = float(calls['q_time'].quantile(0.75))
Q1 = float(calls['q_time'].quantile(0.25))
RIC = Q3 - Q1

Min_q_bp = Q1 - 1.5*RIC
if Min_q_bp < min(calls['q_time']):
    Min_q_bp = min(calls['q_time'])
Max_q_bp = Q3 + 1.5*RIC
if Max_q_bp > max(calls['q_time']):
    Max_q_bp = max(calls['q_time'])

Min_q_bp, Max_q_bp

In [ ]:
#Box plot 
plt.boxplot(calls['q_time']/60)
plt.title('Boxplot de Tiempo en Cola',fontsize=15,fontweight='bold')
plt.ylabel('tiempos (min)')
plt.show()

In [ ]:
#Box plot enfocado en el box
plt.boxplot(calls['q_time']/60)
plt.title('Boxplot de Tiempo en Cola reducido',fontsize=15,fontweight='bold')
plt.ylabel('tiempos (min)')
plt.ylim(Min_q_bp*0.9/60,Max_q_bp*1.1/60)
plt.show()

#### Sigmas

Limites segun el metodo de los 3 sigmas: tiempo maximo 421.61 segundos (7.03 minutos)

In [ ]:
Max_q_s = calls['q_time'].mean() + (3 * calls['q_time'].std())
Min_q_s = calls['q_time'].mean() - (3 * calls['q_time'].std())

if Min_q_s < min(calls['q_time']):
    Min_q_s = min(calls['q_time'])
if Max_q_s > max(calls['q_time']):
    Max_q_s = max(calls['q_time'])

Min_q_s, Max_q_s

#### Conclusion

In [ ]:
#Grafica de los datos sin outliers
plt.plot(calls['q_time']/60,'o', markersize = 0.8)
plt.axhline(y=15, color='green', linestyle='--', lw=1.4)
plt.axhline(y=12, color='purple', linestyle='--', lw=1.4)
plt.axhline(y=Max_q_s/60, color='r', linestyle='--', lw=1.4)
plt.axhline(y=Max_q_bp/60, color='orange', linestyle='--', lw=1.4)
plt.axhline(y=(calls['q_time'].mean())/60, color='black', linestyle='--', lw=1.4)
plt.ylim(0,25)
plt.legend(['Llamadas','15 min', '12 min', '3 sigmas', 'BoxPlot', 'Promedio'],loc="upper right" )
plt.ylabel('Tiempo en minutos')
plt.xlabel('Registros de llamadas')
plt.title('Outliers Tiempo en Cola',fontsize=15,fontweight='bold')
plt.show()

Se realizó el calculo de los limites de tiempo usando Boxplot y los 3 sigmas, sin embargo se consideró que algunos registros que excedian estos limites eran relevantes para el analisis. Teniendo en cuenta el contexto del origen de los datos se determinó un tiempo límite de 15 minutos. De acuerdo con esto, se eliminaron 216 registros (0.05%).

In [40]:
calls.drop(calls[(calls['q_time']<0)|(calls['q_time']>15*60)].index,inplace=True)

Quedaron 433929 registros

### Tiempo en Servicio

In [ ]:
calls['ser_time'].describe()

In [ ]:
plt.plot(calls['ser_time']/60,'o',markersize=0.8)
plt.title('Tiempo en Servicio', fontweight = 'bold',fontsize=15)
plt.xlabel('Llamadas')
plt.ylabel('tiempo (min)')
plt.show()

#### Boxplot

Limites segun el Boxplot: tiempo maximo 440 segundos (7.33 minutos) y minimo 0 segundos <br>

In [ ]:
#Calculo de bigotes
Q3 = float(calls['ser_time'].quantile(0.75))
Q1 = float(calls['ser_time'].quantile(0.25))
RIC = Q3 - Q1

Min_ser_bp = Q1 - 1.5*RIC
if Min_ser_bp < min(calls['ser_time']):
    Min_ser_bp = min(calls['ser_time'])
Max_ser_bp = Q3 + 1.5*RIC
if Max_ser_bp > max(calls['ser_time']):
    Max_ser_bp = max(calls['ser_time'])

Min_ser_bp, Max_ser_bp

In [ ]:
#Box plot completo
plt.boxplot(calls['ser_time']/60)
plt.title('Boxplot de Tiempo en Servicio',fontsize=15,fontweight='bold')
plt.ylabel('tiempos (min)')
plt.show()

In [ ]:
#Box plot enfocado en el box
plt.boxplot(calls['ser_time']/60)
plt.title('Boxplot de Tiempo en Servicio reducido',fontsize=15,fontweight='bold')
plt.ylabel('tiempos (min)')
plt.ylim(Min_ser_bp*0.9/60,Max_ser_bp*1.1/60)
plt.show()

#### Sigmas

Limites segun el metodo de los 3 sigmas: tiempo maximo 951.55 segundos (15.86 minutos) y minimo 0 segundos

In [ ]:
Max_ser_s = calls['ser_time'].mean() + (3 * calls['ser_time'].std())
Min_ser_s = calls['ser_time'].mean() - (3 * calls['ser_time'].std())

if Min_ser_s < min(calls['ser_time']):
    Min_ser_s = min(calls['ser_time'])
if Max_ser_s > max(calls['ser_time']):
    Max_ser_s = max(calls['ser_time'])

Min_ser_s, Max_ser_s, Min_ser_s/60, Max_ser_s/60

#### Conclusion

In [ ]:
#Grafica de los datos
plt.plot(calls['ser_time']/60,'o', markersize = 0.8)
plt.axhline(y=50, color='green', linestyle='--', lw=1.4)
plt.axhline(y=35, color='purple', linestyle='--', lw=1.4)
plt.axhline(y=Max_ser_s/60, color='r', linestyle='--', lw=1.4)
plt.axhline(y=Max_ser_bp/60, color='orange', linestyle='--', lw=1.4) 
plt.axhline(y=calls['ser_time'].mean()/60, color='black', linestyle='--', lw=1.4)
plt.legend(['Llamadas','50 min', '35 min', '3 sigmas', 'BoxPlot', 'Promedio'],loc="upper right" )
plt.ylabel('Tiempo en minutos')
plt.xlabel('Registros de llamadas')
plt.ylim(Min_ser_s*0.9,90)
plt.title('Outliers Tiempo en Servicio',fontsize=15,fontweight='bold')
plt.show()

Se eliminaron 230 registros (0.05%)

In [48]:
calls.drop(calls[(calls['ser_time']<0)|(calls['ser_time']>50*60)].index,inplace=True)

Quedaron 433699 registros

## Analisis de Variables

### Histogramas

In [ ]:
#Histograma de los tiempos
fig, ax = plt.subplots(1, 3,figsize = (15,4.5))
fig.suptitle("Histograma de los tiempos SIN outliers",fontsize=14, fontweight='bold')
ax[0].hist(calls['vru_time'],bins=20,rwidth=0.5)
ax[0].set_title('Tiempo en VRU (seg)')
ax[0].set_xlim(Min_vru_s1,Max_vru_s1)
ax[1].hist(calls['q_time'],bins=35,rwidth=0.5)
ax[1].set_title('Tiempo en cola (seg)')
ax[1].set_xlim(0,15*60)
ax[2].hist(calls['ser_time'],bins=35,rwidth=0.5)
ax[2].set_title('Tiempo en servicio (seg)')
ax[2].set_xlim(0,50*60)
plt.show()

In [ ]:
counts, bins, patches = plt.hist(calls['outcome'],bins=3, rwidth=0.8)

# Agregar los valores encima de cada barra
for count, bin in zip(counts, bins[:-1]):
    plt.text(bin + (bins[1] - bins[0]) / 2, count, str(int(count)), 
           ha='center', va='bottom', fontsize=10)
    
plt.show()

In [ ]:
counts, bins, patches = plt.hist(calls['type'],bins=7, rwidth=0.8)

# Agregar los valores encima de cada barra
for count, bin in zip(counts, bins[:-1]):
    plt.text(bin + (bins[1] - bins[0]) / 2, count, str(int(count)), 
           ha='center', va='bottom', fontsize=10)
    
plt.show()

In [ ]:
counts, bins, patches = plt.hist(calls['priority'],bins=2, rwidth=0.8)

# Agregar los valores encima de cada barra
for count, bin in zip(counts, bins[:-1]):
    plt.text(bin + (bins[1] - bins[0]) / 2, count, str(int(count)), 
           ha='center', va='bottom', fontsize=10)
    
plt.show()

In [ ]:
plt.pie(calls['priority'].value_counts(),labels=calls['priority'].value_counts().index,autopct='%1.1f%%')
plt.show()

In [ ]:
plt.pie(calls['outcome'].value_counts(),labels=calls['outcome'].value_counts().index,autopct='%1.1f%%')
plt.show()

In [ ]:
plt.pie(calls['type'].value_counts(),autopct='%1.1f%%')
plt.legend(['PS - Actividad Regular',
            'NW - Cliente potencial',
            'NE - Actividad por Acciones',
            'IN - Actividad / Consulta por internet',
            'TT - Mensaje'
            'PE - Actividad Regular en inglés',




],loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
plt.show()

In [ ]:
calls['type'].value_counts()
